In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

In [2]:
clmns = [
  "Open 1", "Open 2", "Open 3", "Open 4", "Open 5", "Open 6", "Open 7", "Open 8", "Open 9", "Open 10", "Open 11", "Open 12", "Open 13", "Open 14", "Open 15", "Open 16", "Open 17", "Open 18", "Open 19", "Open 20", "Open 21", "Open 22", "Open 23", "Open 24", "Open 25", "Open 26", "Open 27", "Open 28", "Open 29", "Open 30", 
  "High 1", "High 2", "High 3", "High 4", "High 5", "High 6", "High 7", "High 8", "High 9", "High 10", "High 11", "High 12", "High 13", "High 14", "High 15", "High 16", "High 17", "High 18", "High 19", "High 20", "High 21", "High 22", "High 23", "High 24", "High 25", "High 26", "High 27", "High 28", "High 29", "High 30",
  "Low 1", "Low 2", "Low 3", "Low 4", "Low 5", "Low 6", "Low 7", "Low 8", "Low 9", "Low 10", "Low 11", "Low 12", "Low 13", "Low 14", "Low 15", "Low 16", "Low 17", "Low 18", "Low 19", "Low 20", "Low 21", "Low 22", "Low 23", "Low 24", "Low 25", "Low 26", "Low 27", "Low 28", "Low 29", "Low 30",
  "Close 1", "Close 2", "Close 3", "Close 4", "Close 5", "Close 6", "Close 7", "Close 8", "Close 9", "Close 10", "Close 11", "Close 12", "Close 13", "Close 14", "Close 15", "Close 16", "Close 17", "Close 18", "Close 19", "Close 20", "Close 21", "Close 22", "Close 23", "Close 24", "Close 25", "Close 26", "Close 27", "Close 28", "Close 29", "Close 30",
  "suggestion"]
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["buy","sell"], axis=1)
  y = data[["buy","sell"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1,random_state=99)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-10,10))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(31, data.shape[0]-1):
        grow = []
        ggrow = []
        gggrow = []
        ggggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][0] - data[i-(x-1)][0]] )
            ggrow.append([data[i-x][1] - data[i-(x-1)][1]] )
            gggrow.append([data[i-x][2] - data[i-(x-1)][2]] )
            ggggrow.append([data[i-x][3] - data[i-(x-1)][3]] )

        sugg = "sell"
        if data[i][3] > data[i][0]:
            sugg = "buy"
        arr = np.array(grow).flatten()
        arr2 = np.array(ggrow).flatten()
        arr3 = np.array(gggrow).flatten()
        arr4 = np.array(ggggrow).flatten()

        arr = scaler(arr.reshape(-1, 1))
        arr2 = scaler(arr2.reshape(-1, 1))
        arr3 = scaler(arr3.reshape(-1, 1))
        arr4 = scaler(arr4.reshape(-1, 1))

        arr = np.concatenate((arr, arr2, arr3, arr4), axis=0).reshape(-1,1)
        arr = np.append(arr, sugg)
        row.append(arr)
    grow = []
    ggrow = []
    gggrow = []
    arr = []
    return np.array(row)

In [3]:
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
           if data.shape[0] > 30:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        try:
           unattached_dfs.append(process(pd.read_csv(address)))
        except:
          print(" EP ")
     if np.array(unattached_dfs[0]).shape[0] == 0:
            print(" Null ")
            data = np.array(unattached_dfs[1])
            for z in unattached_dfs[2:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     else:
            data = np.array(unattached_dfs[0])
            for z in unattached_dfs[1:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     unattached_dfs = []
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")      

In [20]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"100d","1h")

Symbols : 1500
Data Folder Removed
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [ ]:
folder_name = extract_data(100)
to_par(folder_name,10)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

In [4]:
xTrain,xTest,yTrain,yTest = spliting(pd.read_parquet("/content/drive/MyDrive/data.parquet"))

(2940686, 120) (2940686, 2)
(326743, 120) (326743, 2)


In [5]:
model = Sequential()

model.add(Dense(1024, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))



opt = tf.keras.optimizers.Adamax()
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              123904    
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_2 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_3 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_4 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_5 (Dense)             (None, 2)                 2050      
                                                                 
Total params: 4,324,354
Trainable params: 4,324,354
Non-

In [6]:
model = load_model("HOG122_9670.h5")

In [7]:
model.fit(xTrain,yTrain,epochs=3,batch_size=1024,validation_data=(xTest,yTest))

Epoch 1/3
2872/2872 [==============================] - 91s 31ms/step - loss: 0.0558 - accuracy: 0.9777 - val_loss: 0.1277 - val_accuracy: 0.9606
Epoch 2/3
2872/2872 [==============================] - 86s 30ms/step - loss: 0.0574 - accuracy: 0.9770 - val_loss: 0.1253 - val_accuracy: 0.9608
Epoch 3/3
2872/2872 [==============================] - 88s 31ms/step - loss: 0.0551 - accuracy: 0.9780 - val_loss: 0.1237 - val_accuracy: 0.9603


In [10]:
model.save("HOG122_9670.h5")

In [7]:
yf.download("btc-usd",period="30d",interval="1h")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
2022-04-09 00:00:00+00:00,42294.703125,42433.007812,42285.593750,42433.007812,42433.007812,0
2022-04-09 01:00:00+00:00,42446.722656,42472.070312,42343.445312,42404.140625,42404.140625,0
2022-04-09 02:00:00+00:00,42431.281250,42442.773438,42225.066406,42225.066406,42225.066406,0
2022-04-09 03:00:00+00:00,42217.382812,42438.730469,42183.253906,42417.109375,42417.109375,416466944
2022-04-09 04:00:00+00:00,42409.160156,42532.078125,42372.222656,42456.734375,42456.734375,186609664
...,...,...,...,...,...,...
2022-05-08 12:00:00+00:00,34784.156250,34818.761719,34632.125000,34644.054688,34644.054688,145690624
2022-05-08 13:00:00+00:00,34645.781250,34661.250000,34443.843750,34443.843750,34443.843750,341708800
2022-05-08 14:00:00+00:00,34445.277344,34552.015625,34395.136719,34466.621094,34466.621094,1075826688
2022-05-08 15:00:00+00:00,34465.386719,34538.480469,34465.386719,34510.832031,34510.832031,109846528


In [8]:
def process_for_prediction(data):
        i = -2
        if len(pd.DataFrame(data).columns) == 7:
          data = data.iloc[: , 1:]        
        data = np.array(data)
        grow = []
        ggrow = []
        gggrow = []
        ggggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][0] - data[i-(x-1)][0]] )
            ggrow.append([data[i-x][1] - data[i-(x-1)][1]] )
            gggrow.append([data[i-x][2] - data[i-(x-1)][2]] )
            ggggrow.append([data[i-x][3] - data[i-(x-1)][3]] )

        arr = np.array(grow).flatten()
        arr2 = np.array(ggrow).flatten()
        arr3 = np.array(gggrow).flatten()
        arr4 = np.array(ggggrow).flatten()

        arr = scaler(arr.reshape(-1, 1))
        arr2 = scaler(arr2.reshape(-1, 1))
        arr3 = scaler(arr3.reshape(-1, 1))
        arr4 = scaler(arr4.reshape(-1, 1))

        arr = np.concatenate((arr, arr2, arr3, arr4), axis=0).reshape(-1,1)
        return arr
def make_prediction_for_yf(symbol,period,timeframe):
    raw_data = process_for_prediction(yf.download(symbol,period=period,interval=timeframe))
    return  model.predict(np.array(raw_data).reshape(1,-1))

In [9]:
make_prediction_for_yf("btc-usd","30d","1h")

[*********************100%***********************]  1 of 1 completed


array([[9.9995184e-01, 7.2661867e-05]], dtype=float32)

In [11]:
def process_for_prediction(data):
        i = -1
        row = []
        data.drop("symbol",axis=1,inplace=True) 
        data = np.array(data)
        grow = []
        ggrow = []
        gggrow = []
        ggggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][0] - data[i-(x-1)][0]] )
            ggrow.append([data[i-x][1] - data[i-(x-1)][1]] )
            gggrow.append([data[i-x][2] - data[i-(x-1)][2]] )
            ggggrow.append([data[i-x][3] - data[i-(x-1)][3]] )

        arr = np.array(grow).flatten()
        arr2 = np.array(ggrow).flatten()
        arr3 = np.array(gggrow).flatten()
        arr4 = np.array(ggggrow).flatten()

        arr = scaler(arr.reshape(-1, 1))
        arr2 = scaler(arr2.reshape(-1, 1))
        arr3 = scaler(arr3.reshape(-1, 1))
        arr4 = scaler(arr4.reshape(-1, 1))

        arr = np.concatenate((arr, arr2, arr3, arr4), axis=0).reshape(-1,1)
        return arr
def make_prediction_for_tv(symbol,timeframe):
   tv = TvDatafeed()
   data = tv.get_hist(symbol='btcUSD',exchange='bitstamp',interval=Interval.in_1_hour,n_bars=100)
   return model.predict(np.array(process_for_prediction(data)).reshape(1,-1))

In [12]:
make_prediction_for_tv("s","x")

you are using nologin method, data you access may be limited


array([[0.9977216 , 0.00254014]], dtype=float32)